# Using Woodwork with Dask and Koalas DataFrames

Woodwork enables DataTables to be created from Dask DataFrames or Koalas DataFrames when working with datasets that are too large to easily fit in memory. Although creating a DataTable from a Dask or Koalas DataFrame follows the same process as one would follow when creating a DataTable from a pandas DataFrame, there are a few limitations to be aware of. This guide will provide a brief overview of creating a DataTable starting with a Dask or Koalas DataFrame, and will outline several key items to keep in mind when using this approach.

Creating DataTables with either Dask or Koalas requires the installation of the Dask or Koalas libraries respectively, which can be installed directly with either of the following commands:

```python
python -m pip install "woodwork[dask]"
```

```python
python -m pip install "woodwork[koalas]"
```

## Dask DataTable Example
First we will create a Dask DataFrame to use in our example. Normally you would create the DataFrame directly by reading in the data from saved files, but we will create it from a demo pandas DataFrame.

In [ ]:
import dask.dataframe as dd
import woodwork as ww

df_pandas = ww.demo.load_retail(nrows=1000, return_dataframe=True)
df_dask = dd.from_pandas(df_pandas, npartitions=10)
df_dask

Now that we have a Dask DataFrame, we can use it to create a Woodwork DataTable, just as we would with a pandas DataFrame:

In [ ]:
dt = ww.DataTable(df_dask, index='order_product_id')
dt

As you can see from the output above, the DataTable was created successfully, and logical type inference was performed for all of the columns. However, this brings us to one of the key issues in working with Dask DataFrames. 

In order to perform logical type inference, Woodwork needs to bring the data into memory so it can be analyzed. Currently, Woodwork reads data from the first partition of data only, and then uses this data for type inference. Depending on the complexity of the data, this could be a time consuming operation. Additionally, if the first partition is not representative of the entire dataset, the logical types for some columns may be inferred incorrectly.

### Skipping or Overriding Type Inference
If this process takes too much time, or if the logical types are not inferred correctly, users have the ability to manually specify the logical types for each column. If the logical type for a column is specified, type inference for that column will be skipped. If logical types are specified for all columns, logical type inference will be skipped completely and Woodwork will not need to bring any of the data into memory when creating the DataTable.

To skip logical type inference completely, and/or to correct type inference issues, you would simply define a logical types dictionary with the correct logical type defined for each column in the dataframe. Then, pass that dictionary to the call to create the DataTable as shown below:

In [ ]:
logical_types = {
    'order_product_id': 'Integer',
    'order_id': 'Categorical',
    'product_id': 'Categorical',
    'description': 'NaturalLanguage',
    'quantity': 'Integer',
    'order_date': 'Datetime',
    'unit_price': 'Double',
    'customer_name': 'FullName',
    'country': 'Categorical',
    'total': 'Double',
    'cancelled': 'Boolean',
}

dt = ww.DataTable(df_dask, index='order_product_id', logical_types=logical_types)
dt

### Analyzing Underlying Data
There are three DataTable methods that also require bringing the underlying Dask DataFrame into memory: `describe`, `value_counts` and `mutual_information`. When called, these methods will call a `compute` operation on the DataFrame associated with the DataTable in order to calculate the desired information. This may be problematic for datasets that cannot fit in memory, so exercise caution when using these methods.

In [ ]:
dt.describe(include=['numeric'])

In [ ]:
dt.value_counts()

In [ ]:
dt.mutual_information().head()

## Koalas DataTable Example
As we did with Dask above, we will first create a Koalas DataFrame to use in our example. Normally you would create the DataFrame directly by reading in the data from saved files, but we will create it from a demo pandas DataFrame.

In [ ]:
import databricks.koalas as ks

df_koalas = ks.from_pandas(df_pandas)
df_koalas.head()

Now that we have a Koalas DataFrame, we can use it to create a Woodwork DataTable, just as we would with a pandas DataFrame:

In [ ]:
dt = ww.DataTable(df_koalas, index='order_product_id')
dt

As you can see from the output above, the DataTable was created successfully, and logical type inference was performed for all of the columns.

### Notes on Koalas Dtype Conversions
In the types table above, one important thing to notice is that the physical types for the Koalas DataTable are quite different than the physical types for the Dask DataTable. The reason for this is that several of the physical types that can be used with pandas and Dask cannot be used with Koalas.

When a Woodwork DataTable is created, the dtype of the underlying dataframe columns are converted to a set of standard dtypes, defined by the LogicalType `pandas_dtype` property. Woodwork uses several nullable dtypes such as `Int64`, `string` and `boolean` which are not compatible with Koalas. In addition the `category` dtype cannot be used with Koalas DataFrames.

For LogicalTypes that have `pandas_dtype` properties that are not compatible with Koalas, Woodwork will try to convert the column dtype, but will be unsuccessful. At that point, Woodwork will then use a backup dtype that is compatible with Koalas. The implication of this is that creating a DataTable from a Koalas DataFrame may result in dtype values that are different than the values you would get when creating the DataTable from an identical pandas DataFrame.

The backup datatypes used are as follows:
`Int64`: `int64`
`boolean`: `bool`
`string`: `object`
`category`: `object`

Since Koalas does not support the `category` dtype, any column that is inferred or specified with a logical type of `Categorical` will have its values converted to strings and stored with a dtype of `object`. This means that a categorical column containing numeric values, will be converted into the equivalent string values when the DataTable is created.

As Koalas does not support the nullable `boolean` dtype, Woodwook will fall back to use the compatible `bool` dtype for columns with a `Boolean` LogicalType. If the corresponding column contains any missing values, these missing values will be converted to `False` values when the column dtype is updated.

Finally, Koalas does not support the `timedelta64[ns]` dtype. For this, there is not a clean backup dtype, so the use of `Timedelta` LogicalType is not supported in Koalas DataTables.

### Skipping or Overriding Type Inference
As with Dask, Woodwork needs to bring the data into memory so it can be analyzed for type inference. Currently, Woodwork uses data from the first 100,000 rows of data only when using a Koalas DataFrame as input, and then uses this data for type inference. If the first 100,000 rows are not representative of the entire dataset, the logical types for some columns may be inferred incorrectly.

To skip logical type inference completely, and/or to correct type inference issues, simply define a logical types dictionary with the correct logical type defined for each column in the dataframe:

In [ ]:
logical_types = {
    'order_product_id': 'Integer',
    'order_id': 'Categorical',
    'product_id': 'Categorical',
    'description': 'NaturalLanguage',
    'quantity': 'Integer',
    'order_date': 'Datetime',
    'unit_price': 'Double',
    'customer_name': 'FullName',
    'country': 'Categorical',
    'total': 'Double',
    'cancelled': 'Boolean',
}

dt = ww.DataTable(df_koalas, index='order_product_id', logical_types=logical_types)
dt

### Analyzing Underlying Data
As with Dask, running `describe`, `value_counts` or `mutual_information` requires bringing the data into memory to perform the analysis. When called, these methods will call a `to_pandas` operation on the DataFrame associated with the DataTable in order to calculate the desired information. This may be problematic for very large datasets, so exercise caution when using these methods.

In [ ]:
dt.describe(include=['numeric'])

In [ ]:
dt.value_counts()

In [ ]:
dt.mutual_information().head()

## Data Validation Limitations

When creating a DataTable several validation checks are performed to confirm that the data in the underlying dataframe is appropriate for the specified parameters. Because some of these validation steps require pulling the underlying data into memory, they are skipped when creating a DataTable from a Dask or Koalas DataFrame. This section provides an overview of the validation checks that are performed with pandas input but skipped with Dask or Koalas input.

### Index Uniqueness
Normally a check is performed to verify that any column specified as the index contains no duplicate values. With Dask or Koalas input, this check is skipped and users must manually verify that any column specified as an index column contains unique values.

### Data Consistency with LogicalType (Dask Only)
If users manually define the LogicalType for a column when creating the DataTable, a check is performed to verify that the data in that column is appropriate for the specified LogicalType. For example, with pandas input if the user specifies a LogicalType of `Double` for a column that contains letters such as `['a', 'b', 'c']`, an error would be raised as it is not possible to convert the letters into numeric values with the `float` dtype associated with the `Double` LogicalType.

With Dask input, no such error would be raised at the time of DataTable creation. However, behind the scenes, Woodwork will have attempted to convert the column physical type to `float`, and this conversion would be added to the Dask task graph, without raising an error. However, an error will be raised if a `compute` operation is called on the underlying DataFrame once Dask attempts to execute the conversion step. Extra care should be taken when using Dask input to make sure any specified logical types are consistent with the data in the columns to avoid this type of error.

### Ordinal Order Values Check
For the `Ordinal` LogicalType, a check is typically performed to make sure that the data column does not contain any values that are not present in the defined order values. This check will not be performed with Dask or Koalas input. Users should manually verify that the defined order values are complete to avoid unexpected results.

## Other Limitations

### Reading from CSV Files
Woodwork provides the ability to read data directly from a CSV file into a DataTable, and during this process Woodwork creates the underlying dataframe so the user does not have to do so. The helper function used for this, `woodwork.read_csv`, will currently only read the data into a pandas DataFrame. At some point, we hope to remove this limitation and also allow data to be read into a Dask or Koalas DataFrame, but for now only pandas DataFrames can be created by this function.

### Sorting DataFrame on Time Index
When creating a DataTable with a time index, Woodwork by default will sort the input dataframe first on the time and then on the index, if specified. Because sorting a distributed DataFrame is a computationally expensive operation, this sorting is performed only when creating a DataTable from a pandas DataFrame. If a sorted DataFrame is needed when using a Dask or Koalas, the user should first sort the DataFrame before creating the DataTable.

### Equality of DataTables
In order to avoid bringing a Dask DataFrame into memory, Woodwoork will not consider the equality of the underlying data when checking whether a DataTable made from a Dask or Koalas DataFrame is equal to another DataTable. This means that two DataTables with identical names, columns, indices, semantic tags, and LogicalTypes but different underlying data will return `True` if at least one of them uses Dask or Koalas.

### LatLong Columns
When working with the LatLong logical type, Woodwork converts all LatLong columns to a standard format of a tuple of strings for Dask dataframes and a list of strings for Koalas dataframes. In order to do this, we bring the data into memory, which may be problematic for large datatsets.